## III -Exploratory Analysis
### A - HACRP Table
#### 1) Importing libraries

In [2]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

#### 2) Querying data from the server

In [ ]:
sql = """
SELECT * FROM hacrp;"""
try:
    dbConnection = psycopg2.connect(
    user = "dap",
    password = "dap",
    host = "83.212.82.56",
    port = "5432",
    database = "dap_medicare")
    hacrp_df = sqlio.read_sql_query(sql, dbConnection) 
except (Exception, psycopg2.Error) as dbError:
    print ("Error while fetching db hacrp:", dbError)
finally:
    if(dbConnection): dbConnection.close()

#### 3) Getting to know the data

In [ ]:
# list the data types for each column
hacrp_df.dtypes
hacrp_df.describe(include=“all”) #summary of all cols
hacrp_df.info #frst 30 & last 30 items
#finding correlations between columns
hacrp_df.corr()


#### 4) Visualising the data- continuous


In [ ]:

sns.regplot(x="w_z score", y="hac_score", data=hacrp_df)
plt.ylim(0,)

#### 4) Visualising the data- categorical



In [ ]:
sns.boxplot(x="state", y="total_hac", data=hacrp_df)